In [11]:
import MySQLdb
import pandas
import itertools
import numpy as np
import bm25fe
import pickle
import subprocess
import jsd

In [12]:
conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugfeature')
cur = conn.cursor()

sql = '''SELECT *
FROM bugs_cpdplatform_ff'''

# bugs = pandas.io.sql.read_sql(sql, conn).set_index(['bug_id'])
bugs = pandas.io.sql.read_sql(sql, conn)

In [13]:
from gensim import corpora
from gensim import matutils

# dictionary = corpora.Dictionary(list(bugs_train['text']))
# dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n = 100000)
dictionary = corpora.Dictionary.load_from_text('dictionary.txt')
num_terms = len(dictionary)
print num_terms

# bugs['summary'] = bugs['short_desc']
bugs['text'] = (bugs['short_desc'] +' '+ bugs['long_desc']).map(lambda x: dictionary.doc2bow(x.split()))
bugs.loc[:,'short_desc'] = bugs['short_desc'].map(lambda x: matutils.corpus2dense([dictionary.doc2bow(x.split())], num_terms, 1)[:,0])
bugs.loc[:,'long_desc'] = bugs['long_desc'].map(lambda x: matutils.corpus2dense([dictionary.doc2bow(x.split())], num_terms, 1)[:,0])

8647


In [14]:
# print bugs_train['short_desc'][0].sum()
# print bugs_train['short_desc'].as_matrix()[4].sum()
# print bugs_train['short_desc'].as_matrix().shape

In [15]:
appearance = np.array(list(bugs['text'].map(lambda x: matutils.corpus2dense([x], num_terms, 1)[:,0]>0)))
df = appearance.sum(0)
idf = np.log(bugs.shape[0]/df)
avgfl = np.array([np.array(list(bugs['short_desc'])).sum(1).mean(), np.array(list(bugs['long_desc'])).sum(1).mean()])

bugs = bugs.set_index(['bug_id'])

In [16]:
# print appearance[1].sum()
# bugs.columns
# print bugs.index[0]

In [32]:
# from gensim.models import hdpmodel
# topicmodel = hdpmodel.HdpModel.load('hdpmodel.txt')
from gensim.models import ldamulticore
topicmodel = ldamulticore.LdaMulticore.load('ldamodel.txt')

# with open('params.txt', 'rb') as f:
#     params = pickle.load(f)
# bm = bm25fe.bm25fe(K1=params[0], d_B=(params[1], params[2]), d_W = (params[3], params[4]), K3=params[5], q_B=(params[6], params[7]), q_W=(params[8], params[9]))
bm = bm25fe.bm25fe(K1=1.2, d_B=(0.75, 0.75), d_W = (2, 1), K3=1.2, q_B=(0.75, 0.75), q_W=(2, 1))

with open('test.txt', 'rb') as f:
    test = pickle.load(f)

num_query = len(test)
num_hit = 0

results = []
for item in test:
    bugs['score'] = bugs.apply(lambda x: bm.score(idf, avgfl, [x[13], x[14]],[bugs.loc[item['query'],'short_desc'], bugs.loc[item['query'],'long_desc']]), axis = 1)
    bugs_sorted = bugs.sort(['score'], ascending = False).iloc[:100]
    idx = 0
    lines = []
    
    for idx in xrange(100):
        sim_title = bugs_sorted.iloc[idx]['short_desc'][bugs.loc[item['query'],'short_desc']>0].sum()/max(bugs_sorted.iloc[idx]['short_desc'].sum(), 1)
        score = bugs_sorted.iloc[idx]['score']
        # cluster = topicmodel.inference([bugs_sorted.iloc[idx]['text'], bugs.loc[item['query'],'text']])
        cluster = topicmodel.inference([bugs_sorted.iloc[idx]['text'], bugs.loc[item['query'],'text']])[0]
        dis_topic = jsd.JSD(cluster[0], cluster[1])
        sim_hos = False
        if (bugs_sorted.iloc[idx]['host_op_sys'] == bugs.loc[item['query'],'host_op_sys']) and (bugs_sorted.iloc[idx]['host_op_sys'] != 'Unknown'):
            sim_hos = True
        sim_gos = False
        if (bugs_sorted.iloc[idx]['guest_op_sys'] == bugs.loc[item['query'],'guest_op_sys']) and (bugs_sorted.iloc[idx]['guest_op_sys'] != 'Unknown'):
            sim_gos = True
        sim_pd = False
        if (bugs_sorted.iloc[idx]['product_id'] == bugs.loc[item['query'],'product_id']):
            sim_pd = True
        sim_cg = False
        if (bugs_sorted.iloc[idx]['category_id'] == bugs.loc[item['query'],'category_id']):
            sim_cg = True
        sim_cp = False
        if (bugs_sorted.iloc[idx]['component_id'] == bugs.loc[item['query'],'component_id']):
            sim_cp = True
        lines.append(str(0)+' qid:'+str(item['query'])+' 1:'+str(sim_title)+' 2:'+str(score)+' 3:'+str(dis_topic)+' 4:'+str(int(sim_hos))+' 5:'+str(int(sim_gos))+' 6:'+str(int(sim_pd))+' 7:'+str(int(sim_cg))+' 8:'+str(int(sim_cp))+' # '+str(bugs.index[idx])+'\n')

    f = open('query.txt', 'wb')
    f.writelines(lines)
    f.close()
            
    subprocess.call(('java', '-jar', 'RankLib-2.1-patched.jar', '-load', 'LambdaMART.txt', '-rank', 'query.txt', '-score', 'score.txt'))
    # subprocess.call(('java', '-jar', 'RankLib-2.1-patched.jar', '-load', 'AdaRank.txt', '-rank', 'query.txt', '-score', 'score.txt'))
    # subprocess.call(('java', '-jar', 'RankLib-2.1-patched.jar', '-load', 'RankNet.txt', '-rank', 'query.txt', '-score', 'score.txt'))
    
    score_rank = []
    f = open('score.txt', 'rb')
    for line in f:
        score_rank.append(float(line.split()[2]))
    f.close()
    
    bugs_sorted['score_rank'] = score_rank
    bugs_ranked = bugs_sorted.sort(['score_rank'], ascending = False)
    
    ranklist = []
    i = 0
    # while len(ranklist) < 5:
    #     if bugs_ranked.index[i] == item['query']:
    #         i += 1
    #     else:
    #         ranklist.append(bugs_ranked.index[i])
    #         i += 1
    while len(ranklist) < 5:
        if bugs_ranked.index[i] != item['query']:
            # child = False
            # for j in xrange(len(ranklist)):
            #     if bugs_ranked.iloc[i]['summary'] == bugs_ranked.loc[ranklist[j],'summary']:
            #         child = True
            #         break
            # if not child:
            #     ranklist.append(bugs_ranked.index[i])
            ranklist.append(bugs_ranked.index[i])
        i += 1
    if len(set(ranklist) & set(item['rel'])) > 0:
        num_hit +=1
    
    results.append('query:'+str(item['query'])+'    '+'dups:'+','.join([str(bug) for bug in item['rel']])+'    '+'ranklist:'+','.join([str(bug) for bug in ranklist])+'\n')
    
print float(num_hit)/num_query
f = open('results.txt', 'wb')
f.writelines(results)
f.close()
        

0.0


In [58]:
# print item
# print test
# print i
# print bugs_ranked
# print bugs['score']
print bugs.iloc[0,13]
for item in test:
    print item
print item['query']
print bugs.loc[item['query']]
print bm.score(idf, avgfl, [bugs.iloc[0,13], bugs.iloc[0,14]], [bugs.loc[item['query'],'short_desc'], bugs.loc[item['query'],'long_desc']])
# print bugs.iloc[0,13].sum()
# print bugs.iloc[0,14].sum()
# for item in idf:
#     print item
# print idf, avgfl
# print bm.fields
# print bm.K1, bm.d_B, bm.d_W, bm.K3, bm.q_B, bm.q_W

[ 0.  0.  0. ...,  0.  0.  0.]
{'query': 1214704, 'rel': [1149327]}
{'query': 290423, 'rel': [289577]}
{'query': 1312084, 'rel': [1298794, 1359716]}
{'query': 350697, 'rel': [350655]}
{'query': 624350, 'rel': [704605]}
{'query': 1134225, 'rel': [1133897]}
{'query': 1134480, 'rel': [1133897]}
{'query': 355000, 'rel': [354998]}
{'query': 1310271, 'rel': [1298794]}
{'query': 1421209, 'rel': [1245375]}
{'query': 1337397, 'rel': [1312931]}
{'query': 1030982, 'rel': [1028935]}
{'query': 1158677, 'rel': [1071227]}
{'query': 1255705, 'rel': [1232621]}
{'query': 824880, 'rel': [1074199]}
{'query': 714889, 'rel': [654800]}
{'query': 1396488, 'rel': [1297619]}
{'query': 787325, 'rel': [714814]}
{'query': 1134192, 'rel': [1133865]}
{'query': 1051529, 'rel': [1055948]}
{'query': 1392674, 'rel': [1261456]}
{'query': 299452, 'rel': [296424]}
{'query': 1134126, 'rel': [1133962]}
{'query': 661557, 'rel': [636113]}
{'query': 158781, 'rel': [158787]}
{'query': 601323, 'rel': [604024]}
{'query': 692089, '

In [21]:
print num_hit,num_query
# ranklist = []
# print len(ranklist)
# print bugs_sorted['score']
# print bugs_ranked['score_rank']

0 42


In [13]:
# with open('params.txt', 'rb') as f:
#     params = pickle.load(f)
# bm = bm25fe.bm25fe(K1=params[0], d_B=(params[1], params[2]), d_W = (params[3], params[4]), K3=params[5], q_B=(params[6], params[7]), q_W=(params[8], params[9]))
bm = bm25fe.bm25fe(K1=1.2, d_B=(0.75, 0.75), d_W = (2, 1), K3=1.2, q_B=(0.75, 0.75), q_W=(2, 1))
hit5 = 0
hit10 = 0
for item in test:
    bugs['score'] = bugs.apply(lambda x: bm.score(idf, avgfl, [x[1], x[7]],[bugs.loc[item['query'],'short_desc'], bugs.loc[item['query'],'long_desc']]), axis = 1)
    bugs_sorted = bugs.sort(['score'], ascending = False).iloc[:100]
    lst5 = []
    i = 0
    while len(lst5) < 5:
        if bugs_sorted.index[i] == item['query']:
            i += 1
        else:
            lst5.append(bugs_sorted.index[i])
            i += 1
    if len(set(lst5) & set(item['rel'])) > 0:
        hit5 +=1
    lst10 = []
    i = 0
    while len(lst10) < 10:
        if bugs_sorted.index[i] == item['query']:
            i += 1
        else:
            lst10.append(bugs_sorted.index[i])
            i += 1
    if len(set(lst10) & set(item['rel'])) > 0:
        hit10 +=1

print float(hit5)/num_query
print float(hit10)/num_query

0.595238095238
0.690476190476


In [43]:
# print bugs_sorted.iloc[0]

print test[0]['rel']

[1149327]


In [73]:
# bugs_train

In [74]:
# print bugs_train['guest_op_sys']

In [100]:
print len(lines)
print lines[10053]


10100
0 qid:1224296 1:0.142857 2:77.9385298109 3:0.0788403087743 4:1 5:0 6:1 7:0 8:0 # 1425012



In [87]:
print bugs_train.loc[485186,'short_desc'].sum()

0.0
